In [107]:
# Predicting Mortality by Heart Failure

## Import Libraries

In [108]:
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense

## Import Data

In [109]:
data = pd.read_csv('heart_failure_clinical_records_dataset.csv')
data.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## Inspect Data

In [110]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 30.5 KB


In [111]:
# Distribution of DEATH_EVENT
Counter(data['DEATH_EVENT'])

Counter({0: 203, 1: 96})

## Extract Features

In [112]:
# y
y = data['DEATH_EVENT']

# x
x_features = []

for i in data.columns:
    if i != 'DEATH_EVENT':
        x_features.append(i)

x = data[x_features]

In [113]:
x = pd.get_dummies(x)

In [114]:
# Split x and y in training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(x, 
                                                    y, 
                                                    train_size=0.8,
                                                   test_size=0.2,
                                                   random_state=40)

In [115]:
# Standardise x variables
ct = ColumnTransformer([('numeric', StandardScaler(), ['age',
                                                     'creatinine_phosphokinase',
                                                     'ejection_fraction',
                                                     'platelets',
                                                     'serum_creatinine',
                                                     'serum_sodium',
                                                     'time'])])
X_train = ct.fit_transform(X_train)
X_test = ct.fit_transform(X_test)

In [119]:
# Prepare labels for classification
le = LabelEncoder()

Y_train = le.fit_transform(Y_train.astype(str))
Y_test = le.fit_transform(Y_test.astype(str))

Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

ValueError: y should be a 1d array, got an array of shape (239, 2) instead.

## Model

### Design the Model

In [117]:
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1])))
model.add(Dense(12, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Fit the Model

In [118]:
model.fit(X_train, Y_train, epochs=100, batch_size=16)

Epoch 1/100


ValueError: in user code:

    File "/Users/charlottedriver/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Users/charlottedriver/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/charlottedriver/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Users/charlottedriver/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/charlottedriver/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/Users/charlottedriver/anaconda3/lib/python3.11/si